In [50]:
import torch
from torch import nn

import torchvision.datasets as datasets
from torchvision.transforms import ToTensor

In [51]:
# Defining the device = cuda to run the code on gpu
if torch.cuda.is_available():
    device = torch.device("cuda")

In [52]:
# Download mnist fashion train and test datasets
mnist_train = datasets.FashionMNIST(root="./data", download=True, train=True, transform=ToTensor())
mnist_test = datasets.FashionMNIST(root="./data", download=False, train=False, transform=ToTensor())

In [53]:
from torch.utils.data import DataLoader

# Converting to dataloaders
train_dataloder = DataLoader(mnist_train, batch_size=32, shuffle=True)
test_dataloder = DataLoader(mnist_test, batch_size=32, shuffle=True)

In [54]:
# Defining the model with inputs at shape(1, 28, 28)
model = nn.Sequential(
    nn.Sequential(
        nn.Conv2d(1, 32, kernel_size=(3, 3), padding=1, padding_mode="reflect"), # CNN 2 dimensional layer with a 3x3 filter - Shape(32, 28, 28)
        nn.MaxPool2d(kernel_size=2), # Max pooling to reduce the dimension and turn less complex the nn - Shape(32, 14, 14)
        nn.BatchNorm2d(32), # Normalizates data to improve accuracy
        nn.ReLU(),
        nn.Dropout(0.1) # Dropout random connections between neurons to improve accuracy
    ),

    nn.Sequential(
        nn.Conv2d(32, 64, kernel_size=(3, 3), padding=1, padding_mode="reflect"), # Shape(64, 14, 14)
        nn.MaxPool2d(kernel_size=2), # Shape(64, 7, 7)
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.Dropout(0.1)
    ),
    
    nn.Flatten(), # Shape(3136) = 64 * 7 * 7
    
    nn.Sequential(
        nn.Linear(3136, 1000),
        nn.BatchNorm1d(1000),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(1000, 100),
        nn.BatchNorm1d(100),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(100, 10)
    )
).to(device)

# Defining the loss function
criterion = nn.CrossEntropyLoss()

# Defining the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [55]:
# Reshaping the data (validation)
for X, y in train_dataloder:
    print(X.shape)
    print(y.shape)
    
    y = nn.functional.one_hot(y, num_classes=10).type(torch.float32)
    print(y.shape)
    break

torch.Size([32, 1, 28, 28])
torch.Size([32])
torch.Size([32, 10])


In [56]:
for i in range(10):
    loss_sum = 0
    for X, y in train_dataloder:
        X = X.to(device)
        y = nn.functional.one_hot(y, num_classes=10).type(torch.float32).to(device)

        optimizer.zero_grad()
        y_pred = model(X)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()

    print(loss_sum)

800.9967646300793
566.9247032441199
492.1472059749067
443.8024645783007
403.50094435736537
371.8289270736277
350.2957790195942
326.1061526173726
303.34149074880406
283.908728832379


In [57]:
model.eval()
with torch.no_grad():
    accurate = 0
    total = 0
    for X, y in test_dataloder:
        X = X.to(device)
        y = y.to(device)

        outputs = nn.functional.softmax(model(X), dim=1) 
        correct_pred = (y == outputs.max(dim=1).indices)

        total+=correct_pred.size(0)
        accurate+=correct_pred.type(torch.int).sum().item()

    print(f"Accuracy: {(accurate/total)*100:.2f} %")

Accuracy: 92.77 %
